# Estudo com a série histórica dos sorteios da MS

### Estudo simples para avaliação de probabilidade de acerto de um sorteio. Não existe uma receita ou resposta exata, mas sim os resultados mais prováveis, ou seja, sem causalidade ou casualidade. Simplesmente, probabilidade.

### Após extrair a série histórica, vamos estruturar os dados com as informações que nos interessam

In [34]:
import json
import datetime
import pprint as pp

In [3]:
# ler o arquivo dump dos resultados
file = "bucket/contest_results.raw"
with open(file, 'r') as reader:
    resp = reader.read()
    
results = "[" + resp.replace("\n",", ") + "]" # gambs... :)
results = json.loads(results)
results

[{'proximoConcurso': '2289',
  'concursoAnterior': '2288',
  'forward': None,
  'mensagens': [],
  'concurso': 2289,
  'data': 1597287600000,
  'resultado': '06-34-37-38-45-09',
  'ganhadores': 0,
  'ganhadores_quina': 33,
  'ganhadores_quadra': 2808,
  'valor': 0.0,
  'valor_quina': 53948.07,
  'valor_quadra': 905.72,
  'acumulado': 1,
  'valor_acumulado': 23126358.55,
  'dtinclusao': 1597364040000,
  'prox_final_zero': '2290',
  'ac_final_zero': 23126358.55,
  'proxConcursoFinal': None,
  'observacao': None,
  'rowguid': 'B2AC2104-E618-4A51-9B41-8E8CD1B0CE8C',
  'ic_conferido': '1',
  'de_local_sorteio': 'Espaço Loterias Caixa',
  'no_cidade': 'SÃO PAULO',
  'sg_uf': 'SP',
  'vr_estimativa': 27000000.0,
  'dt_proximo_concurso': 1597460400000,
  'vr_acumulado_especial': 53260061.41,
  'vr_arrecadado': 30877992.0,
  'ic_concurso_especial': False,
  'error': False,
  'rateioProcessamento': False,
  'sorteioAcumulado': True,
  'concursoEspecial': '0',
  'ganhadoresPorUf': None,
  'result

In [57]:
# mapear campos que interessam
def mapKeys(obj):
    newObj = {}
    newObj['concurso'] = obj['concurso']
    newObj['data'] = datetime.datetime.fromtimestamp( int(str(obj['data']).zfill(13)[0:10]) ).strftime("%Y-%m-%d")
    newObj['resultado'] = obj['resultado'].split('-')
    newObj['valor'] = obj['valor']
    newObj['valor_quina'] = obj['valor_quina']
    newObj['valor_quadra'] = obj['valor_quadra']
    newObj['ganhadores'] = obj['ganhadores']
    newObj['ganhadores_quina'] = obj['ganhadores_quina']
    newObj['ganhadores_quadra'] = obj['ganhadores_quadra']
    newObj['acumulado'] = obj['acumulado']
    newObj['valor_acumulado'] = obj['valor_acumulado']
    newObj['vr_arrecadado'] = obj['vr_arrecadado']
    newObj['vr_acumulado_especial'] = obj['vr_acumulado_especial']
    #newObj['sg_uf'] = obj['sg_uf']
    #newObj['no_cidade'] = obj['no_cidade']
    newObj['ganhadoresPorUf'] = list(map(lambda x: {"uf": x['sgUf'], "cidade": x['noCidade'], "qtGanhadores": x['qtGanhadores']}, obj['ganhadoresPorUf'])) if obj['ganhadoresPorUf'] is not None else None

    return newObj

In [58]:
# guardar os dados para análise em uma lista
df_list = []
for o in results:
    df_list.append(mapKeys(o))
    
pp.pprint(df_list)

[{'acumulado': 1,
  'concurso': 2289,
  'data': '2020-08-13',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 2808,
  'ganhadores_quina': 33,
  'resultado': ['06', '34', '37', '38', '45', '09'],
  'valor': 0.0,
  'valor_acumulado': 23126358.55,
  'valor_quadra': 905.72,
  'valor_quina': 53948.07,
  'vr_acumulado_especial': 53260061.41,
  'vr_arrecadado': 30877992.0},
 {'acumulado': 1,
  'concurso': 2288,
  'data': '2020-08-11',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 1545,
  'ganhadores_quina': 16,
  'resultado': ['35', '39', '02', '26', '56', '40'],
  'valor': 0.0,
  'valor_acumulado': 9101162.4,
  'valor_quadra': 1205.49,
  'valor_quina': 81484.2,
  'vr_acumulado_especial': 52791565.07,
  'vr_arrecadado': 22612711.5},
 {'acumulado': 1,
  'concurso': 2287,
  'data': '2020-08-08',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 3261,
  'ganhadores_quina': 45,
  'resultado': ['29', '04', '02', '41', '06', '56'],
  'v

  'resultado': ['01', '37', '47', '44', '46', '32'],
  'valor': 0.0,
  'valor_acumulado': 27369478.22,
  'valor_quadra': 1274.09,
  'valor_quina': 81317.28,
  'vr_acumulado_especial': 2681650.14,
  'vr_arrecadado': 47953575.0},
 {'acumulado': 1,
  'concurso': 2224,
  'data': '2020-01-15',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 2283,
  'ganhadores_quina': 28,
  'resultado': ['23', '50', '58', '32', '52', '16'],
  'valor': 0.0,
  'valor_acumulado': 22276448.91,
  'valor_quadra': 1372.37,
  'valor_quina': 78328.49,
  'vr_acumulado_especial': 1954074.51,
  'vr_arrecadado': 38039692.5},
 {'acumulado': 1,
  'concurso': 2223,
  'data': '2020-01-11',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 2702,
  'ganhadores_quina': 45,
  'resultado': ['49', '26', '02', '56', '40', '42'],
  'valor': 0.0,
  'valor_acumulado': 9638420.62,
  'valor_quadra': 1132.5,
  'valor_quina': 47600.35,
  'vr_acumulado_especial': 1376917.27,
  'vr_arrecadado': 37151

  'vr_acumulado_especial': 60227351.64,
  'vr_arrecadado': 34500333.0},
 {'acumulado': 1,
  'concurso': 2173,
  'data': '2019-07-27',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 3884,
  'ganhadores_quina': 51,
  'resultado': ['09', '48', '44', '42', '02', '50'],
  'valor': 0.0,
  'valor_acumulado': 6019424.06,
  'valor_quadra': 673.28,
  'valor_quina': 35892.64,
  'vr_acumulado_especial': 59703903.99,
  'vr_arrecadado': 31749879.0},
 {'acumulado': 1,
  'concurso': 2172,
  'data': '2019-07-24',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 3356,
  'ganhadores_quina': 73,
  'resultado': ['09', '57', '43', '37', '28', '24'],
  'valor': 0.0,
  'valor_acumulado': 2647404.63,
  'valor_quadra': 611.76,
  'valor_quina': 19687.16,
  'vr_acumulado_especial': 59222186.91,
  'vr_arrecadado': 24927133.0},
 {'acumulado': 0,
  'concurso': 2171,
  'data': '2019-07-20',
  'ganhadores': 1,
  'ganhadoresPorUf': [{'cidade': 'SÃO PAULO', 'qtGanhadores': 1, 'u

  'ganhadoresPorUf': None,
  'ganhadores_quadra': 4938,
  'ganhadores_quina': 92,
  'resultado': ['27', '21', '03', '11', '15', '49'],
  'valor': 0.0,
  'valor_acumulado': 2691007.02,
  'valor_quadra': 422.62,
  'valor_quina': 15878.62,
  'vr_acumulado_especial': 6354575.76,
  'vr_arrecadado': 25337679.5},
 {'acumulado': 0,
  'concurso': 2121,
  'data': '2019-02-02',
  'ganhadores': 1,
  'ganhadoresPorUf': [{'cidade': 'RIO DE JANEIRO',
                       'qtGanhadores': 1,
                       'uf': 'RJ'}],
  'ganhadores_quadra': 5493,
  'ganhadores_quina': 82,
  'resultado': ['37', '08', '17', '29', '10', '40'],
  'valor': 24666686.76,
  'valor_acumulado': 0.0,
  'valor_quadra': 682.17,
  'valor_quina': 31988.03,
  'vr_acumulado_especial': 5970146.17,
  'vr_arrecadado': 45495432.5},
 {'acumulado': 1,
  'concurso': 2120,
  'data': '2019-01-30',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 3897,
  'ganhadores_quina': 61,
  'resultado': ['38', '41', '20', '2

  'valor_acumulado': 49363979.62,
  'valor_quadra': 789.19,
  'valor_quina': 39904.7,
  'vr_acumulado_especial': 37516533.48,
  'vr_arrecadado': 53294356.5},
 {'acumulado': 1,
  'concurso': 2058,
  'data': '2018-07-11',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 5435,
  'ganhadores_quina': 81,
  'resultado': ['19', '04', '23', '29', '59', '56'],
  'valor': 0.0,
  'valor_acumulado': 31279809.85,
  'valor_quadra': 650.29,
  'valor_quina': 30543.83,
  'vr_acumulado_especial': 36707938.23,
  'vr_arrecadado': 42911631.0},
 {'acumulado': 1,
  'concurso': 2057,
  'data': '2018-07-07',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 4086,
  'ganhadores_quina': 42,
  'resultado': ['37', '38', '20', '13', '10', '54'],
  'valor': 0.0,
  'valor_acumulado': 26722348.27,
  'valor_quadra': 677.06,
  'valor_quina': 46108.12,
  'vr_acumulado_especial': 36056872.29,
  'vr_arrecadado': 33588695.0},
 {'acumulado': 1,
  'concurso': 2056,
  'data': '2018-07-05'

...

  'ganhadoresPorUf': None,
  'ganhadores_quadra': 8717,
  'ganhadores_quina': 110,
  'resultado': ['19', '18', '52', '25', '17', '49'],
  'valor': 0.0,
  'valor_acumulado': 6697873.66,
  'valor_quadra': 152.31,
  'valor_quina': 12115.7,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 268,
  'data': '2001-04-21',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 10133,
  'ganhadores_quina': 111,
  'resultado': ['21', '10', '47', '06', '05', '33'],
  'valor': 0.0,
  'valor_acumulado': 5098601.74,
  'valor_quadra': 112.93,
  'valor_quina': 10348.29,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 267,
  'data': '2001-04-14',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 4076,
  'ganhadores_quina': 46,
  'resultado': ['58', '01', '45', '55', '08', '46'],
  'valor': 0.0,
  'valor_acumulado': 3720210.04,
  'valor_quadra': 260.19,
  'valor_quina': 23142.59,
  'vr_acu

 {'acumulado': 1,
  'concurso': 206,
  'data': '2000-02-12',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 3478,
  'ganhadores_quina': 43,
  'resultado': ['20', '26', '56', '18', '34', '51'],
  'valor': 0.0,
  'valor_acumulado': 916897.74,
  'valor_quadra': 218.86,
  'valor_quina': 17769.34,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 0,
  'concurso': 205,
  'data': '2000-02-05',
  'ganhadores': 1,
  'ganhadoresPorUf': [{'cidade': None, 'qtGanhadores': 1, 'uf': 'RJ'}],
  'ganhadores_quadra': 6014,
  'ganhadores_quina': 71,
  'resultado': ['26', '32', '54', '43', '46', '40'],
  'valor': 4490339.49,
  'valor_acumulado': 0.0,
  'valor_quadra': 178.86,
  'valor_quina': 15208.03,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 204,
  'data': '2000-01-29',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 2400,
  'ganhadores_quina': 32,
  'resultado': ['31', '41', '51', '39', '5

  'resultado': ['24', '06', '38', '23', '28', '16'],
  'valor': 3805750.38,
  'valor_acumulado': 0.0,
  'valor_quadra': 162.48,
  'valor_quina': 11760.17,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 146,
  'data': '1998-12-19',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 4079,
  'ganhadores_quina': 51,
  'resultado': ['51', '34', '25', '60', '42', '32'],
  'valor': 0.0,
  'valor_acumulado': 2747335.81,
  'valor_quadra': 298.52,
  'valor_quina': 23923.45,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 145,
  'data': '1998-12-12',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 3389,
  'ganhadores_quina': 71,
  'resultado': ['43', '15', '41', '51', '16', '31'],
  'valor': 0.0,
  'valor_acumulado': 1283220.59,
  'valor_quadra': 314.91,
  'valor_quina': 15061.27,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 0,
  'concurso': 14

  'ganhadores_quina': 102,
  'resultado': ['10', '34', '04', '06', '25', '40'],
  'valor': 0.0,
  'valor_acumulado': 1780905.8,
  'valor_quadra': 138.4,
  'valor_quina': 7539.68,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 83,
  'data': '1997-10-05',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 4756,
  'ganhadores_quina': 76,
  'resultado': ['24', '16', '04', '42', '57', '15'],
  'valor': 0.0,
  'valor_acumulado': 858048.93,
  'valor_quadra': 150.04,
  'valor_quina': 9408.43,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 0,
  'concurso': 82,
  'data': '1997-09-28',
  'ganhadores': 1,
  'ganhadoresPorUf': [{'cidade': None, 'qtGanhadores': 1, 'uf': 'BA'}],
  'ganhadores_quadra': 5889,
  'ganhadores_quina': 77,
  'resultado': ['51', '36', '53', '29', '18', '15'],
  'valor': 13877826.02,
  'valor_acumulado': 0.0,
  'valor_quadra': 208.57,
  'valor_quina': 15982.73,
  'vr_acumulado_especial': 

  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 22,
  'data': '1996-08-05',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 4865,
  'ganhadores_quina': 47,
  'resultado': ['09', '38', '56', '01', '31', '46'],
  'valor': 0.0,
  'valor_acumulado': 14716828.84,
  'valor_quadra': 262.9,
  'valor_quina': 27212.9,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 21,
  'data': '1996-07-29',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 5030,
  'ganhadores_quina': 63,
  'resultado': ['49', '46', '06', '53', '33', '36'],
  'valor': 0.0,
  'valor_acumulado': 13182021.44,
  'valor_quadra': 221.62,
  'valor_quina': 17694.19,
  'vr_acumulado_especial': None,
  'vr_arrecadado': None},
 {'acumulado': 1,
  'concurso': 20,
  'data': '1996-07-22',
  'ganhadores': 0,
  'ganhadoresPorUf': None,
  'ganhadores_quadra': 5728,
  'ganhadores_quina': 72,
  'resultado': ['25', '28', '30', '33', '51', '11'],
  'valor':

In [59]:
#############



In [68]:
list( filter(lambda a: a['data'] <= '2020-08-08', df_list) )

[{'concurso': 2287,
  'data': '2020-08-08',
  'resultado': ['29', '04', '02', '41', '06', '56'],
  'valor': 0.0,
  'valor_quina': 45775.06,
  'valor_quadra': 902.38,
  'ganhadores': 0,
  'ganhadores_quina': 45,
  'ganhadores_quadra': 3261,
  'acumulado': 1,
  'valor_acumulado': 6699522.84,
  'vr_arrecadado': 35727336.0,
  'vr_acumulado_especial': 52448473.7,
  'ganhadoresPorUf': None},
 {'concurso': 2286,
  'data': '2020-08-05',
  'resultado': ['42', '43', '41', '09', '21', '30'],
  'valor': 0.0,
  'valor_quina': 56321.66,
  'valor_quadra': 1366.2,
  'ganhadores': 0,
  'ganhadores_quina': 28,
  'ganhadores_quadra': 1649,
  'acumulado': 1,
  'valor_acumulado': 2905011.81,
  'vr_arrecadado': 27352228.5,
  'vr_acumulado_especial': 51906400.68,
  'ganhadoresPorUf': None},
 {'concurso': 2285,
  'data': '2020-08-01',
  'resultado': ['12', '10', '33', '07', '42', '01'],
  'valor': 11323343.69,
  'valor_quina': 15996.75,
  'valor_quadra': 458.94,
  'ganhadores': 2,
  'ganhadores_quina': 160,
 